# Prochiral Michael Acceptors

Here we examine substituted acrylamides as examples of prochiral Michael Acceptors. These are important as the addition reaction can give rise to chiral centres which need to be taken into account. 

In [ ]:
from rdkit.Chem import MolFromSmiles, MolFromMolBlock, MolToSmiles, MolToMolBlock, AddHs, RemoveHs, Mol, SanitizeMol

from rdkit.Chem.AllChem import EmbedMolecule, ReactionFromSmarts

In [ ]:
from rdkit.Chem.rdmolops import AssignStereochemistryFrom3D

from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions

from rdkit.Chem.Draw import MolsToGridImage

In [ ]:
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

PandasTools.RenderImagesInAllDataFrames()
IPythonConsole.ipython_useSVG = True

#### Reaction

The reaction is a combined addition and keto-enol rearrangement...

In [ ]:
reaction = ReactionFromSmarts('[O:1]=[C,S:2]-[C:3]=[C:4].[Sv2H:5]>>[O:1]=[C,S:2]-[C:3]-[C:4]-[S:5]')

reaction

Note that the reaction could also be fully specified, _i.e._ with H counts made explicit. This can have advantages, but means distinct versions would be necessary for the various substitution patterns. The version below would only work for unsubsubstituted acceptors, for example.

The advantage of using a fully specified reaction is that it reaction works even if stereo is assigned from 3D and the mol is not recreated from a string.

In [ ]:
# reaction = ReactionFromSmarts('[O:1]=[C,S:2]-[CH:3]=[CH2:4].[Sv2H1:5]>>[O:1]=[C,S:2]-[CH2:3]-[CH2:4]-[SH0:5]')

# reaction

Thiol with an identifiable tag that be removed easily later...

In [ ]:
tagged_thiol = MolFromSmiles('S[At]')

#### Input with assigned chiral centre

In [ ]:
# Assign stero from SMILES...

smiles = '[C@H](Br)(I)NC(=O)\C(C)=C(/Cl)F'  # Acetamide is Prochiral

# smiles = '[C@H](Br)(I)NC(=O)C=C'  # Acetamide is not prochiral

mol = MolFromSmiles(smiles)

print(MolToSmiles(mol))

mol

In [ ]:
# # Assign stereo from 3D...
# # N.B. This will be an arbitrary (random) enantiomer unless a seed is specified for the embedding step.

# smiles = 'C(Br)(I)NC(=O)C(C)=C(Cl)F'  # Acetamide is Prochiral

# # smiles = 'C(Br)(I)NC(=O)C=C'  # Acetamide is not prochiral

# mol = MolFromSmiles(smiles)

# mol = AddHs(mol)  # Hs are required for 3D structure generation (also copies mol)

# assert EmbedMolecule(mol) != -1, "Error! The 3D embedding of the molecule failed."

# AssignStereochemistryFrom3D(mol)

# mol.Compute2DCoords()  # For depiction

# # mol = RemoveHs(mol)  # Redundant if mol recreated from string

# # Recreate mol from string seems to be required for reaction to work if stereo assigned from 3D...

# mol = MolFromSmiles(MolToSmiles(mol))  # MolBlock also works

# # mol = MolFromMolBlock(MolToMolBlock(mol))  # SMILES also works

# # mol = Mol(mol)  # N.B. Copying mol doesn't work

# print(MolToSmiles(mol))

# mol

#### Perform the reaction

In [ ]:
products = reaction.RunReactants((mol, tagged_thiol))

new_mol = products[0][0]

SanitizeMol(new_mol)

print(MolToSmiles(new_mol))

new_mol

#### Enumerate the new chiral centres

https://www.rdkit.org/docs/source/rdkit.Chem.EnumerateStereoisomers.html

Note that, by default, the code only expands unspecified stereocenters.

_N.B._ Recreating the product mol from string (SMILES or MolBlock) seems to be required for stereo enumeration to work (again, a deep copy of the mol doesn't work)

In [ ]:
assert len(tuple(EnumerateStereoisomers(new_mol))) == 1

new_mol = MolFromMolBlock(MolToMolBlock(new_mol))  # SMILES also works

# # mol = Mol(mol)  # N.B. Copying mol doesn't work

mols = tuple(EnumerateStereoisomers(new_mol))

assert len(mols) > 1  # If input was prochiral!

MolsToGridImage(mols, molsPerRow=4, useSVG=True)